# Genie Deep Dive: Making Your Semantic Model Conversational

Now that you've created a semantic model with metric views, it's time to make it conversational with Genie. Leadership wants you to enable natural language queries so anyone can ask questions about the IoT data without writing SQL.

## What You'll Learn

✅ Create a Genie Space for your semantic model  
✅ Add instructions to guide query generation  
✅ Define filters for common questions  
✅ Test with a benchmark query  

---

## The Scenario

Your semantic model (`iot_sensor_metrics`) is ready, but the operations team doesn't know SQL. They need to ask questions like:
- "What's the average temperature in the West region?"
- "Show me devices with high anomaly rates"
- "Which factory has the most critical readings?"

Genie will translate these natural language questions into queries against your semantic model.

---

## What is Genie?

**Genie** is Databricks' AI-powered natural language interface for querying data. It uses your semantic model and additional context to generate accurate SQL queries.

**Key Components for This Exercise:**
- **Genie Space**: Container pointing to your metric view
- **Instructions**: Business context and terminology
- **Filters**: Pre-defined query shortcuts
- **Benchmark**: Test question to verify accuracy

---

**References:**
- [Genie Setup](https://docs.databricks.com/aws/en/genie/set-up)
- [Best Practices](https://docs.databricks.com/aws/en/genie/best-practices)

## Step 1: Create Your Genie Space  

### Setup in the UI

**Create a Semantic Model Genie:**
1. Click **Genie** in the sidebar
2. Click **New**
3. Select **Metric views**
4. Select `dwx_airops_insights_platform_dev_workspace.{username}.iot_sensor_metrics` (the metric view you created in notebook 1)

**Why use the metric view?**
- ✅ Pre-defined metrics are already understood by Genie
- ✅ Consistent business logic
- ✅ Faster queries (uses aggregated data)
- ✅ Better accuracy (clear metric definitions)

---

**⚠️ Keep it simple**: Just the metric view is enough to get started. You can add more tables later if needed.

## Step 2: Add Instructions

### What are Instructions?

**Instructions** teach Genie about your business context and terminology. They're shown to Genie when generating queries.

### Add Business Context

- In your Genie Space, click **Instructions** and add this Text:

```
You are helping analyze data from aircraft sensors deployed across models.

### Business Rules
- Critical threshold: Temperature > 85°F
- Normal operating range: 60-75°F
```
- Click Save  


**Why this helps:**
- ✅ Knows what thresholds matter
- ✅ Generates more accurate queries

---

**💡 Tip**: Keep instructions concise. Too much information can confuse Genie.

## Step 3: Define Example Queries

### Add Common Filters

In your Genie Space Instructions tab, select **SQL Queries** and add this example:

**What are our at-risk devices?:**
```
SELECT 
  `Device ID`,
  `Model ID`,
  `Factory ID`,
  `Date`,
  MEASURE(`Max Temperature`) AS `Max Temperature`,
  MEASURE(`Record Count`) AS `Record Count`,
  MEASURE(`Total Delay`) AS `Total Delay`,
  MEASURE(`Average Airflow Rate`) AS `Average Airflow Rate`
FROM sensor_metrics 
GROUP BY ALL
HAVING `Max Temperature` > 85
ORDER BY `Max Temperature` DESC;
```


With this example defined, Genie will understand what "at-risk" is in the context of our business and apply the correct SQL query.


## Step 4: Add a join key

Flip back to the Data tab
Click + add
Select the dim_models table
Click Instructions tab
Click Joins
Set Left Table to sensor_metrics
Set Right Table to dim_models
Join condition `Model ID` = `model_id`
Relationship Type: Many to One
Instructions: get more information about the models from the metric view

## Step 5: Test with a Benchmark

### What is a Benchmark?

A **benchmark** is a test question used to verify Genie is generating accurate queries. It helps you measure and improve quality.

### Create Your First Benchmark

In your Genie Space, navigate to **Benchmarks** and add this test question:

```
Question: What was the most common model category for at-risk devices?

Expected Answer Type: One model category and the total number of at risk devices

Why this tests multiple features:
- Uses the metric view (average_temperature)
- Requires a join to the other table to get model category
- Tests that Genie understands your definition of 'at-risk'
```

### Run the Benchmark

1. Click **Run Benchmark**
2. Review the generated SQL
3. Check if the answer is reasonable

---

### Improve if Needed

If the benchmark fails add more context to fix that mistake!

**Success Criteria**: Genie generates the correct SQL and returns a reasonable value.


## Summary and Next Steps

### What You Accomplished

✅ **Created a Genie Space** pointing to your semantic model  
✅ **Added instructions** with business context and terminology  
✅ **Defined a join key** for common query patterns  
✅ **Tested with a benchmark** to verify accuracy  

### Your Genie Space is Now Ready!

Users can now ask questions like:
- "What's the average temperature by factory?"
- "What was the most common model_category for at-risk devices?"
- "What are the critical temperatures in the last week?"

No SQL required! Genie translates these to queries against your `iot_sensor_metrics` semantic model.

---

## Try This Out (Optional Extensions)

Want to go further? Try these enhancements:

### 1. Add More Filters
Create filters for:
- Different regions (East, Central, International)
- Time ranges (last 30 days, last month)
- Model categories

### 2. Create Additional Benchmarks
Test more complex scenarios:
- "Which factory has the highest average temperature?"
- "Show me device count trends over time"
- "Compare West vs East regions"

### 3. Add Sample Q&A
In the Instructions, add example questions and expected results to improve accuracy.

### 4. Add SQL Query Examples as Trusted Assets

SQL Query Examples help Genie understand how to write queries for your specific data model. Add them as trusted assets:

**Reference:** [SQL Query Examples Documentation](https://docs.databricks.com/aws/en/genie/trusted-assets)

### 5. Join to More Tables
If you add `dim_factories`, test questions that need factory details:
- "What's the temperature at factories in California?"
- "Show me metrics for factories with more than 100 devices"

---

## Key Takeaways

**Semantic Model + Genie = Self-Service Analytics**
1. **Semantic model** (notebook 1) defines consistent metrics
2. **Genie** (this notebook) makes them conversational
3. **Result**: Non-technical users can explore data

**Start Simple, Iterate**
- Begin with essential context
- Add filters as users need them
- Use benchmarks to measure quality
- Improve based on actual usage


---

**Additional Resources:**
- [Genie Documentation](https://docs.databricks.com/aws/en/genie/)
- [Best Practices](https://docs.databricks.com/aws/en/genie/best-practices)
- [Knowledge Store Guide](https://docs.databricks.com/aws/en/genie/knowledge-store)
